In [ ]:
# load necessary libraries
import os, glob
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
from keras.models import load_model
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import Model



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2025-08-01 14:13:11.209592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# load all models from the specified folder
# assuming the models are saved in Keras format with '.keras' extension
# Say 'B-type' model, where 'B' stands for 'Base'.
folder = 'CIFAR10models/ResNet/'
pattern = os.path.join(folder, '*cifar10*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}


# load additional models from another folder
# assuming these models are also saved in Keras format with '.keras' extension
# Say 'SB-type', where 'SB' stands for 'Specialized Base'.
folder = 'CIFAR10models/more_tunned/'
pattern = os.path.join(folder, '*_more_specialized*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models.update({os.path.basename(f): load_model(f) for f in file_list})

label_dict = {(3,1) : 'ResNet20v1',
        (3,2) : 'ResNet20v2',
        (9,1): 'ResNet56v1',
        (9,2): 'ResNet56v2'}

(x, y), (x_test, y_test) = cifar10.load_data()
x_train, x_v, y_train, y_v = train_test_split(x, y, test_size=0.2, 
                                              random_state=42) # random state has been always 42.
x_train = x_train.astype('float32') / 255.0
x_val = x_v.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train_categorical_10 = to_categorical(y_train,10)
y_val_categorical_10 = to_categorical(y_v,10)
y_test_categorical_10 = to_categorical(y_test,10)
y_test_categorical_20 = to_categorical(y_test,20)



In [3]:

import math
sigmoid = lambda x: 1 / (1 + math.exp(-x))

# analyze the behavior of each model on the test set
print("Analyzing the behavior of each model on the test set...\n")
print(f"Total number of models loaded: {len(loaded_models)} and the size of testing dataset: {len(x_test)}")
print(f"Certainty threshold for predictions: {sigmoid(10)}\n")
results = {}
reports = {}
for file_name, model in loaded_models.items():
    # Extract model name from the file name
    base = file_name.replace('.keras','')
    parts = base.split('_')
    if parts[0] == 'n':
        model_name = label_dict[(int(parts[1]),int(parts[2][-1]))] + '_B_'+parts[-1][-1]
    else:
        model_name = parts[0] + '_SB_' + parts[-1]
    print(f"Model: {model_name}")
    
    prediction = model.predict(x_test,verbose = 0)
    global_acc = np.sum(np.argmax(prediction, axis=-1) == y_test.reshape(-1)) / len(x_test)
    certain_mask = np.max(prediction, axis=-1) > sigmoid(10)
    size_of_certain_part = np.sum(certain_mask)
    accuracy_on_certain = np.sum(np.argmax(prediction[certain_mask],axis=-1) == y_test[certain_mask].reshape(-1))/np.sum(size_of_certain_part)
    print(f"Size of certain part: {size_of_certain_part} (percertage: {size_of_certain_part / len(x_test) * 100:.2f}%)")
    print(f"Accuracy on certain part and global accuracy: {accuracy_on_certain:.4f} and {global_acc:.4f}")
    print("-" * 50)
    results[model_name] = {'global_acc': global_acc, 'acc_on_certain': accuracy_on_certain, 'size_of_certain':size_of_certain_part,
                           'certain_mask': certain_mask, 'prediction': prediction}
    reports[model_name] = {'global_acc': global_acc, 'acc_on_certain': accuracy_on_certain,
                           'size_of_certain': size_of_certain_part}

reports_df = pd.DataFrame(reports).T
reports_df = reports_df.sort_values(by='global_acc', ascending=False)
# Save the results to a file
import pickle as pkl
with open('results/results_behaviour_of_B_and_SB_on_original.pkl', 'wb') as f:
    pkl.dump(results, f)
    

Analyzing the behavior of each model on the test set...

Total number of models loaded: 40 and the size of testing dataset: 10000
Certainty threshold for predictions: 0.9999546021312976

Model: ResNet20v1_B_0
Size of certain part: 3060 (percertage: 30.60%)
Accuracy on certain part and global accuracy: 0.9931 and 0.8248
--------------------------------------------------
Model: ResNet20v1_B_1
Size of certain part: 3405 (percertage: 34.05%)
Accuracy on certain part and global accuracy: 0.9968 and 0.8520
--------------------------------------------------
Model: ResNet20v1_B_2
Size of certain part: 3387 (percertage: 33.87%)
Accuracy on certain part and global accuracy: 0.9976 and 0.8533
--------------------------------------------------
Model: ResNet20v1_B_3
Size of certain part: 3124 (percertage: 31.24%)
Accuracy on certain part and global accuracy: 0.9987 and 0.8592
--------------------------------------------------
Model: ResNet20v1_B_4
Size of certain part: 2117 (percertage: 21.17%)
Acc

In [ ]:
# With sigmoid(10) certainty threshold, on the original CIFAR10 test set
# B and SB models performance are as follows:
reports_df

,global_acc,acc_on_certain,size_of_certain
ResNet56v2_SB_0,0.9203,0.997330,4494.0
ResNet56v2_SB_1,0.9174,0.998816,4222.0
ResNet56v2_SB_3,0.9158,0.997928,4344.0
ResNet56v1_SB_3,0.9128,0.996752,4310.0
ResNet56v1_SB_0,0.9113,0.998525,4069.0
ResNet56v2_SB_2,0.9109,0.998060,4640.0
ResNet20v2_SB_0,0.9101,0.996911,4532.0
ResNet56v1_SB_1,0.9096,0.996934,4566.0
ResNet20v2_SB_2,0.9095,0.998687,3809.0
ResNet20v1_SB_2,0.9093,0.998770,4066.0


In [ ]:
# As SB model shows better performance on the certain part, let use SB type models to analyze the behavior on the certain part against adversarial examples.

# Load the adversarial examples

## set all files.
## Possibly there is no merged testing dataset, or validation dataset.
## We only look for testing dataset.
import re

pattern1 = re.compile(r'_\d+to\d+_')
pattern2 = re.compile(r'test')

folder = './adversarial_examples/gen_by_ResNet'
all_files = os.listdir(folder)

filtered_files = [
    fname for fname in all_files
    if fname.endswith('.npy') and not pattern1.search(fname) and pattern2.search(fname)
]


## load all the testing dataset
## including testing dataset generated by VGG
loaded_testing_dataset = {}
import re
for f in filtered_files:
    
    base = f.replace('.keras.npy','').replace('.npy','')
    parts = base.split('_')
    attack_type = parts[0]
    if attack_type == 'cwl2':
        direction = parts[3]
        if direction == 'targeted':
            direction = direction +'_'+ parts[5]
        if parts[-1].isdigit():
            idx = 1
        else:
            idx = 0
        model_type = 'ResNet56v1'
    else:
        
        direction = parts[4]
        if direction == 'target':
            if parts[6] == 'second':
                direction = 'targeted_2nd'
                model_type = 'ResNet56v1'
                
            else:
                direction = 'targeted_ll'
                model_type = label_dict[(int(parts[10]),
                                        int(parts[11][-1]))]
                idx = parts[-1][-1]
        else:
            direction = 'untargeted'
            model_type = label_dict[(int(parts[8]),
                                        int(parts[9][-1]))]
            idx = parts[-1][-1]
    key = (attack_type, direction, model_type, idx)
    loaded_testing_dataset[key] = np.load(os.path.join(folder, f))


folder = './adversarial_examples/gen_by_VGG'
all_files = os.listdir(folder)

filtered_files = [
    fname for fname in all_files
    if fname.endswith('.npy') and pattern2.search(fname)
]
for f in filtered_files:
    base = f.replace('.npy','')
    parts = base.split('_')
    attack_type = parts[0]
    direction = parts[4]
    if direction == 'target':
        direction = 'targeted_ll'
        model_type = parts[8]
        if parts[-1].isdigit():
            idx = int(parts[-1])
        else:
            idx = 4
    else:
        direction = 'untargeted'
        model_type = parts[6]
        if parts[-1].isdigit():
            idx = int(parts[-1])
        else:
            idx = 4
    key = (attack_type, direction, model_type, idx)
    loaded_testing_dataset[key] = np.load(os.path.join(folder, f))

            
for k in loaded_testing_dataset.keys():
    print(k)

In [11]:
# Recall the results of the B and SB models on the adversarial examples.

avg_of_B_and_SB = pd.read_csv('../results/single_models/avg_of_B_and_SB.csv', index_col=0)

avg_of_B_and_SB_self_attack = pd.read_csv('../results/single_models/avg_of_B_and_SB_self_attack.csv', index_col=0)

avg_of_B_and_SB_self_attack2 = pd.read_csv('../results/single_models/avg_of_B_and_SB_self_attack2.csv', index_col=0)

In [12]:
avg_of_B_and_SB


,B,S-B
pgd_targeted_2nd_ResNet56v1,0.375219,0.561091
pgd_targeted_ll_ResNet56v1,0.506472,0.710215
pgd_targeted_ll_ResNet20v1,0.510594,0.710166
cwl2_untargeted_ResNet56v1,0.513648,0.577115
pgd_targeted_ll_vgg16,0.522184,0.693613
pgd_targeted_ll_vgg19,0.522727,0.692869
pgd_targeted_ll_ResNet56v2,0.526864,0.723670
pgd_targeted_ll_ResNet20v2,0.531371,0.725438
pgd_targeted_ll_vgg13,0.548100,0.705959
pgd_targeted_ll_vgg11,0.654498,0.795841


In [ ]:
# List of folders and (optional) patterns to search for models
model_sources = [
    # ('adversarial_models/full_and_fine/', '*.keras'),
    # ('adversarial_models/specialized_from_full_and_fine/', '*.keras'),
    # ('adversarial_models/naive_method/', '*.keras'),
    ('CIFAR10models/more_tunned/', '*.keras'),
    ('CIFAR10models/ResNet/', '*cifar10*.keras'),
]

# Initialize list of all found files
file_list = []

for folder, pattern in model_sources:
    full_pattern = os.path.join(folder, pattern)
    matched_files = sorted(glob.glob(full_pattern))
    file_list.extend(matched_files)

# # Load all models into dictionary: {filename: model}
# loaded_models = {os.path.basename(f): load_model(f) for f in file_list}

In [ ]:
def get_intersection(committee: list[str], certain_masks : dict = {}) -> np.ndarray:
    """
    Compute the intersection of predictions from a committee of models.
    
    Args:
        committee (list[str]): List of model names in the committee.
        
    Returns:
        np.ndarray: Intersection of certain parts.
    """
    if not certain_masks:
        raise ValueError("certain_masks dictionary is empty. Please provide a valid dictionary.")
    domains = [certain_masks[member] for member in committee if member in certain_masks]
    return np.logical_and.reduce(np.array(domains))

from scipy.stats import entropy

def compute_entropy(committee: list[str], locations : np.ndarray, sample:np.ndarray) -> np.ndarray:
    """
    Compute the entropy of predictions from a committee of models at specified locations.
    
    Args:
        committee (list[str]): List of model names in the committee.
        locations (np.ndarray): Boolean array indicating the locations to compute entropy.
        sample (np.ndarray): Sample data for which to compute entropy.
        
    Returns:
        np.ndarray: Entropy values at the specified locations.
    """
    if not committee:
        return np.zeros_like(locations, dtype=float)
    sample = sample[locations]
    reports = {}
    
    n = np.shape(locations)[0]
    for member in committee:
        p = loaded_models[member].predict(sample, verbose=0)
        certainty = np.max(p,axis=-1)
        classes = np.argmax(p,axis=-1)
        reports[member] = {'certainty': certainty, 'classes': classes}
    entropy_values = np.zeros(n, dtype=float)
    for i in range(n):
        counts = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
        for member in committee:
            counts[reports[member]['classes'][i]] += 1
        entropy_values[i] = entropy(counts, base = 10)
    reports['entropy'] = entropy_values
    return reports
        

In [ ]:
loaded_models = {}
for f in file_list:
    base = os.path.basename(f)
    parts = base.replace('.keras','').split('_')
    if parts[0] == 'classifying':
        model_structure = parts[3]
        model_type = 'S-ff'
        augmented_by = parts[5].split('-')[0]
        idx = parts[-1]
        model_key = (model_structure, model_type, augmented_by, idx)

    elif parts[0] == 'n':
        model_structure = label_dict[(int(parts[1]),
                                    int(parts[2][-1]))]
        model_type = 'B'
        augmented_by = 'None'
        idx = parts[-1][-1]
        model_key = (model_structure, model_type, augmented_by, idx)
    else:
        model_structure = parts[0]
        if parts[1] == 'ff':
            model_type = 'ff'
            augmented_by = parts[2].split('-')[0]
            idx = parts[-1]
            model_key = (model_structure, model_type, augmented_by, idx)
            
        elif parts[1] == 'naive':
            if parts[-1] == 'fine':
                model_type = 'S-nai'
            else:
                model_type = 'nai'
            augemented_by = parts[2]
            idx = 0
            model_key = (model_structure, model_type, augmented_by, idx)
        else:
            if parts[1] == 'more':
                model_type = 'S-B'
                augmented_by = 'None'
                idx = parts[-1]
                model_key = (model_structure, model_type, augmented_by, idx)
            else:
                continue
    
    loaded_models[model_key]=load_model(f)

In [ ]:
committees = [[k for k in loaded_models.keys() if k.startswith('ResNet20v1') and k.split('_')[1] == 'B'],
              [k for k in loaded_models.keys() if k.startswith('ResNet20v2') and k.split('_')[1] == 'B'],
                [k for k in loaded_models.keys() if k.startswith('ResNet56v1') and k.split('_')[1] == 'B'],
                [k for k in loaded_models.keys() if k.startswith('ResNet56v2') and k.split('_')[1] == 'B'],
                [k for k in loaded_models.keys() if k.startswith('ResNet20v1') and k.split('_')[1] == 'SB'],
                [k for k in loaded_models.keys() if k.startswith('ResNet20v2') and k.split('_')[1] == 'SB'],
                [k for k in loaded_models.keys() if k.startswith('ResNet56v1') and k.split('_')[1] == 'SB'],
                [k for k in loaded_models.keys() if k.startswith('ResNet56v2') and k.split('_')[1] == 'SB']]

predictions= {f: loaded_models[f].predict(x_test,verbose=0) for f in loaded_models.keys()}
certain_masks = {f: np.max(predictions[f],axis=-1)>sigmoid(10) for f in loaded_models.keys()}

results = {}

for comittee in committees:
    comittee_name = comittee[0].split('_')[0] + '_' + comittee[0].split('_')[1]
    print(f"Analyzing committee: {comittee_name}")
    results[comittee_name] = {}
    intersection = get_intersection(comittee, certain_masks)
    print(f"Intersection of {comittee} has size: {np.sum(intersection)}")
    if np.sum(intersection) > 0:
        for sample_name, sample in loaded_testing_dataset.items():
            print(f"Computing entropy for {sample_name}...")
            reports = compute_entropy(comittee, intersection, sample)
            print(f"Average entropy of the intersection: {np.mean(reports['entropy'])}")
            results[comittee_name][sample_name] = reports
            
    else:
        print("No intersection found.")

with open('results/results_entropy_of_committees.pkl', 'wb') as f:
    pkl.dump(results, f)


In [ ]:
dict_of_df = {}
for comittee in committees:
    comittee_name = comittee[0][0] + '_' + comittee[0][1]
    dict_of_df[comittee_name] = {}
    for sample_name in loaded_testing_dataset.keys():
        
        rows = []
        report = results[comittee_name][sample_name]
        sample_name_str = str(sample_name[0])+'_' + str(sample_name[1]) + '_' + str(sample_name[2]) + '_' + str(sample_name[3])
        
        for k in report.keys():
            if k != 'entropy':
                model_name = k[0] + '_' + k[1] + '_' + k[3]
#                 c = report[k]['certainty']
                prediction = report[k]['classes']
#                 row = [model_name] + list(np.stack((c,prediction), axis=1))
                row = [model_name] + list(prediction)
            else:
                row = ['entropy'] +list(report[k])
            rows.append(row)
            columns = ['model_name'] + [str(i) for i in range(len(prediction))]
        reports_df = pd.DataFrame(rows, columns=columns)
        dict_of_df[comittee_name][sample_name_str] = reports_df
    

In [ ]:
with open('results/results_dict_of_df_behaviour_on_certain_parts_B_SB_adv.pkl', 'wb') as f:
    pkl.dump(dict_of_df, f)

In [ ]:
for model_name, _ in dict_of_df.keys():
    for sample_name, df in dict_of_df[model_name].items():
        print(f"Model: {model_name}, Sample: {sample_name}")
        print(df.head())
        print("-" * 50)
        print('entropy:', df[df['model_name'] == 'entropy'].values[0][1:])
        print('average entropy:', np.mean(df[df['model_name'] == 'entropy'].values[0][1:]))